In [2]:
import numpy as np
from scipy import stats
import pandas as pd

# 数据：每行包含五个方法的 (均值, 方差) 数据
data = [
    [(-0.1875, 0.0255), (-0.2964, 0.0342), (-15.1641, 182.3778), (-0.0383, 9.2933e-04), (-0.0359, 9.1409e-04)],
    [(-0.1483, 0.0121), (-0.241, 0.0346), (-2.2224, 2.1253), (-0.0337, 0.0010), (-0.0337, 0.0010)],
    [(-0.1104, 0.0067), (-0.1751, 0.0177), (-0.7775, 0.2305), (-0.0301, 9.7874e-04), (-0.034, 0.0036)],
    [(-0.0887, 0.0036), (-0.1327, 0.0068), (-0.3965, 0.0539), (-0.0285, 8.1340e-04), (-0.0278, 8.5759e-04)],
    [(-0.072, 0.0028), (-0.1023, 0.0049), (-0.2337, 0.0220), (-0.0276, 8.0112e-04), (-0.0265, 7.9155e-04)],
    [(-0.0603, 0.0026), (-0.0844, 0.0043), (-0.1602, 0.0108), (-0.0265, 8.9227e-04), (-0.0255, 9.3228e-04)],
    [(-0.0534, 0.0023), (-0.0721, 0.0033), (-0.1194, 0.0058), (-0.0258, 8.7413e-04), (-0.0245, 8.7482e-04)],
    [(-0.0485, 0.0023), (-0.0623, 0.0031), (-0.0941, 0.0050), (-0.0258, 8.6598e-04), (-0.0244, 8.3746e-04)],
    [(-0.0445, 0.0020), (-0.0553, 0.0028), (-0.0782, 0.0042), (-0.0238, 7.9857e-04), (-0.0224, 7.7180e-04)],
    [(-0.0393, 0.0020), (-0.0489, 0.0026), (-0.0664, 0.0038), (-0.0221, 8.0327e-04), (-0.021, 7.7949e-04)],
    [(-0.0355, 0.0018), (-0.0439, 0.0023), (-0.0577, 8.2280e-04), (-0.0214, 8.2280e-04), (-0.0206, 8.0087e-04)],
    [(-0.0323, 0.0016), (-0.0395, 0.0019), (-0.0512, 0.0026), (-0.0201, 7.7535e-04), (-0.0197, 7.6252e-04)],
    [(-0.0296, 0.0014), (-0.0351, 0.0016), (-0.0442, 0.0021), (-0.0191, 7.1829e-04), (-0.0186, 7.0128e-04)],
    [(-0.0288, 0.0012), (-0.0339, 0.0015), (-0.0422, 0.0018), (-0.0192, 6.9300e-04), (-0.0186, 6.7638e-04)],
    [(-0.028, 0.0011), (-0.033, 0.0014), (-0.0409, 0.0017), (-0.0182, 6.2087e-04), (-0.0178, 6.0575e-04)],
    [(-0.0261, 8.6546e-04), (-0.0309, 0.0010), (-0.0381, 0.0013), (-0.0178, 5.7239e-04), (-0.0173, 5.6234e-04)]
]

# 计算单边 t 检验的 t 统计量和 p 值
def one_sided_t_test(mean1, var1, mean2, var2, n=500):
    # 计算标准误差 (SE)
    se1 = np.sqrt(var1 / n)
    se2 = np.sqrt(var2 / n)
    
    # 计算 t 统计量
    t_stat = (mean1 - mean2) / np.sqrt(se1**2 + se2**2)
    
    # 计算 p 值 (注意: 单边检验)
    df = 2 * n - 2  # 自由度
    p_value = 1 - stats.t.cdf(t_stat, df)  # 单边右尾 p 值
    
    return t_stat, p_value

# 准备检验名称
test_names = [
    "SEM > SDE",
    "SEM > LSHE", 
    "SEM > NLSH",
    "SCME > SEM",
    "SCME > SDE",
    "SCME > LSHE",
    "SCME > NLSH"
]

# 准备记录检验结果的列表
results_data = []

# 循环每一行数据，进行七个单边检验
for i, row in enumerate(data):
    # 提取每个方法的均值和方差
    n_values, lshe_values, sde_values, sem_values, scme_values = row
    
    n_mean, n_var = n_values
    lshe_mean, lshe_var = lshe_values
    sde_mean, sde_var = sde_values
    sem_mean, sem_var = sem_values
    scme_mean, scme_var = scme_values
    
    # 对每一行进行七个单边检验
    row_results = {}
    row_results['行号'] = i + 1
    
    # 执行七个检验
    test_results = []
    test_results.append(one_sided_t_test(sem_mean, sem_var, sde_mean, sde_var))  # SEM > SDE
    test_results.append(one_sided_t_test(sem_mean, sem_var, lshe_mean, lshe_var))  # SEM > LSHE
    test_results.append(one_sided_t_test(sem_mean, sem_var, n_mean, n_var))  # SEM > NLSH
    test_results.append(one_sided_t_test(scme_mean, scme_var, sem_mean, sem_var))  # SCME > SEM
    test_results.append(one_sided_t_test(scme_mean, scme_var, sde_mean, sde_var))  # SCME > SDE
    test_results.append(one_sided_t_test(scme_mean, scme_var, lshe_mean, lshe_var))  # SCME > LSHE
    test_results.append(one_sided_t_test(scme_mean, scme_var, n_mean, n_var))  # SCME > NLSH
    
    # 将检验结果添加到当前行
    for j, (t_stat, p_value) in enumerate(test_results):
        test_name = test_names[j]
        row_results[f'{test_name}_t'] = f"{t_stat:.4f}"
        row_results[f'{test_name}_p'] = f"{p_value:.4e}"
        row_results[f'{test_name}_显著'] = "是" if p_value < 0.05 else "否"
    
    results_data.append(row_results)

# 创建DataFrame
df_results = pd.DataFrame(results_data)

# 重新排列列顺序，使相关检验放在一起
column_order = ['行号']
for test_name in test_names:
    column_order.extend([f'{test_name}_t', f'{test_name}_p', f'{test_name}_显著'])

df_results = df_results[column_order]

# 打印表格
print("单边t检验结果汇总表")
print("=" * 100)
print(df_results.to_string(index=False))
print()

# 也可以选择性地输出显著性统计
print("显著性统计:")
print("-" * 50)
for test_name in test_names:
    significant_count = df_results[f'{test_name}_显著'].eq('是').sum()
    total_count = len(df_results)
    print(f"{test_name}: {significant_count}/{total_count} 行显著 (p < 0.05)")
print()

# 保存到CSV文件
df_results.to_csv('t_test_results.csv', index=False, encoding='utf-8-sig')
print("结果已保存到 't_test_results.csv' 文件")

# 可选：创建更紧凑的显示格式
print("\n紧凑格式（仅显示显著性和p值）:")
print("=" * 80)

compact_df = pd.DataFrame()
compact_df['行号'] = df_results['行号']

for test_name in test_names:
    compact_df[test_name] = df_results.apply(
        lambda row: f"t={row[f'{test_name}_t']}, p={row[f'{test_name}_p']}, 显著={row[f'{test_name}_显著']}", 
        axis=1
    )

print(compact_df.to_string(index=False))

单边t检验结果汇总表
 行号 SEM > SDE_t SEM > SDE_p SEM > SDE_显著 SEM > LSHE_t SEM > LSHE_p SEM > LSHE_显著 SEM > NLSH_t SEM > NLSH_p SEM > NLSH_显著 SCME > SEM_t SCME > SEM_p SCME > SEM_显著 SCME > SDE_t SCME > SDE_p SCME > SDE_显著 SCME > LSHE_t SCME > LSHE_p SCME > LSHE_显著 SCME > NLSH_t SCME > NLSH_p SCME > NLSH_显著
  1     25.0447  0.0000e+00            是      30.7920   0.0000e+00             是      20.5216   0.0000e+00             是       1.2499   1.0581e-01             否      25.0487   0.0000e+00             是       31.0851    0.0000e+00              是       20.8577    0.0000e+00              是
  2     33.5629  0.0000e+00            是      24.5674   0.0000e+00             是      22.3890   0.0000e+00             是       0.0000   5.0000e-01             否      33.5629   0.0000e+00             是       24.5674    0.0000e+00              是       22.3890    0.0000e+00              是
  3     34.7362  0.0000e+00            是      23.7235   0.0000e+00             是      20.4906   0.0000e+00             是      -1